# Pre-processing

In [303]:
import pandas as pd
import numpy as np

In [304]:
data_sets = pd.read_csv('./cow.csv')
data_sets.columns

Index(['surgery', 'age', 'hospital_number', 'temperature', 'pulse',
       'respiratory_rate', 'temp_of_extremities', 'peripheral_pulse',
       'mucous_membrane', 'capillary_refill_time', 'pain', 'peristalsis',
       'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux',
       'nasogastric_reflux_ph', 'rectal_exam_feces', 'abdomen',
       'packed_cell_volume', 'total_protein', 'abdomo_appearance',
       'abdomo_protein', 'outcome Class', 'surgical_lesion', 'lesion'],
      dtype='object')

In [329]:
feture_cols = ['surgery', 'age', 'temperature', 'pulse', 'respiratory_rate', 'packed_cell_volume', 'total_protein', 'lesion', 'temp_of_extremities', 'mucous_membrane', 'outcome Class']
df = data_sets[feture_cols]

df.surgery = df.surgery.map({'no' : 0, 'yes' : 1})
df.age = df.age.map({'young' : 0, 'adult' : 1})
df.temp_of_extremities = df.temp_of_extremities.map({'normal':0, 'warm':1, 'cool' : 2, 'cold' : 3})
df.mucous_membrane = df.mucous_membrane.map({'normal_pink': 1, 'bright_pink': 2, 'pale_pink': 3,
                                            'pale_cyanotic':4, 'bright_red': 5, 'dark_cyanotic': 6})
# df.loc[df.peripheral_pulse == 'normal', 'peripheral_pulse'] = 1
# df.loc[df.peripheral_pulse == 'increased', 'peripheral_pulse'] = 2
# df.loc[df.peripheral_pulse == 'reduced', 'peripheral_pulse'] = 3
# df.loc[df.peripheral_pulse == 'absent', 'peripheral_pulse'] = 4

# df.temperature.sum(axis=0)/df.temperature.size
df.loc[df.temperature.isnull(), 'temperature'] = 37.8
df.loc[df.pulse.isnull(), 'pulse'] = 35
# df.loc[df.abdomo_protein.isnull(), 'abdomo_protein'] = 3
df = df.drop(df[df.isnull().sum(axis=1) > 0].index)
df

C:\Users\asus\AppData\Local\Temp\ipykernel_9120\468895310.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.surgery = df.surgery.map({'no' : 0, 'yes' : 1})
C:\Users\asus\AppData\Local\Temp\ipykernel_9120\468895310.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.age = df.age.map({'young' : 0, 'adult' : 1})
C:\Users\asus\AppData\Local\Temp\ipykernel_9120\468895310.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

,surgery,age,temperature,pulse,respiratory_rate,packed_cell_volume,total_protein,lesion,temp_of_extremities,mucous_membrane,outcome Class
2,0,1,38.3,40.0,24.0,33.0,6.7,0,0.0,3.0,lived
3,1,0,39.1,164.0,84.0,48.0,7.2,2208,3.0,6.0,died
6,1,1,37.9,48.0,16.0,37.0,7.0,3124,0.0,1.0,lived
8,0,1,37.8,80.0,36.0,38.0,6.2,3205,2.0,3.0,euthanized
10,1,1,38.1,66.0,12.0,44.0,6.0,2124,2.0,5.0,lived
...,...,...,...,...,...,...,...,...,...,...,...
292,1,1,37.8,78.0,24.0,43.0,62.0,2209,2.0,3.0,euthanized
293,0,1,38.5,40.0,16.0,37.0,67.0,0,0.0,1.0,lived
294,1,1,37.8,120.0,70.0,55.0,65.0,3205,3.0,4.0,euthanized
296,1,1,37.5,72.0,30.0,60.0,6.8,3205,3.0,4.0,died


In [306]:
[i for i in np.array(data_sets.columns) if i not in np.array(feture_cols)]

['hospital_number',
 'peripheral_pulse',
 'capillary_refill_time',
 'pain',
 'peristalsis',
 'abdominal_distention',
 'nasogastric_tube',
 'nasogastric_reflux',
 'nasogastric_reflux_ph',
 'rectal_exam_feces',
 'abdomen',
 'abdomo_appearance',
 'abdomo_protein',
 'surgical_lesion']

In [307]:
df['outcome Class'].value_counts()

lived         113
died           45
euthanized     21
Name: outcome Class, dtype: int64

In [308]:
X = df
y = X['outcome Class'].values
X = X.drop(['outcome Class'], axis=1)

In [309]:
y[y=='lived'] = 1
y[y=='died'] = 2
y[y=='euthanized'] = 3
y = y.astype('int')

# แยกข้อมูลฝึก/ทดสอบ

In [319]:
# split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [320]:
y_train = y_train.astype('int')

In [321]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = { 'criterion':['gini','entropy'], 'max_depth': np.arange(3, 100)}
dtree_model = DecisionTreeClassifier()
dtree_gscv = GridSearchCV(dtree_model, param_grid, cv=10)
dtree_gscv.fit(X_train, y_train)
dtree_gscv.best_params_

{'criterion': 'entropy', 'max_depth': 4}

In [322]:
dtree_gscv.best_score_

0.7276190476190476

In [323]:
y_pred = dtree_gscv.predict(X_test)
np.sum(y_pred == y_test)/y_test.size

0.6944444444444444

# use oversampling using SMOTE to increase data for training set 

In [315]:
from imblearn .over_sampling import SMOTE

sm = SMOTE()
X_resampled, y_resampled = sm.fit_resample(X_train,y_train)

In [316]:
param_grid = { 'criterion':['gini','entropy'], 'max_depth': np.arange(3, 100)}
dtree_gscv = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=10)
dtree_gscv.fit(X_resampled, y_resampled)
dtree_gscv.best_params_

{'criterion': 'entropy', 'max_depth': 45}

In [317]:
dtree_gscv.best_score_

0.8106153846153846

In [318]:
y_pred = dtree_gscv.best_estimator_.predict(X_test)
np.sum(y_pred == y_test)/y_test.size

0.5555555555555556